In [2]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported')

!conda install -c conda-forge folium=0.5.0 --yes
import folium #ploting library

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatiom #might not work

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

Libraries imported
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NE

ImportError: cannot import name 'Nominatiom'

## Scraping wiki page and making panadas data frame

In [4]:
#downloading wiki as read_html
import pandas as pd
df_wiki=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df_wiki)

    Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
9        M8A      Not assigned   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
13       M2B      Not assigned   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
20       M7B      Not assigned   
21       M8B      Not assigned   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C  

In [10]:
#drop borough
dfwikid=df_wiki[df_wiki.Borough != 'Not assigned']

#more than 1 neighbor hood in postal code? combine the two neighbor rows into one with 
    #neighborhoods separated via comma
df2nb=dfwikid.groupby("Postcode").agg(lambda x:','.join(set(x)))
print(df2nb)


                   Borough                                      Neighbourhood
Postcode                                                                     
M1B            Scarborough                                      Rouge,Malvern
M1C            Scarborough               Rouge Hill,Highland Creek,Port Union
M1E            Scarborough                    West Hill,Guildwood,Morningside
M1G            Scarborough                                             Woburn
M1H            Scarborough                                          Cedarbrae
M1J            Scarborough                                Scarborough Village
M1K            Scarborough          Kennedy Park,Ionview,East Birchmount Park
M1L            Scarborough                      Clairlea,Golden Mile,Oakridge
M1M            Scarborough      Cliffside,Cliffcrest,Scarborough Village West
M1N            Scarborough                         Birch Cliff,Cliffside West
M1P            Scarborough  Wexford Heights,Dorset Park,Scarboro

In [13]:
#borough but no neighbohod? neighborhood is same as borough
df2nb.loc[df2nb['Neighbourhood']== 'Not assigned','Neighbourhood']=df2nb.loc[df2nb['Neighbourhood']== 'Not assigned','Borough']
print(df2nb)

                   Borough                                      Neighbourhood
Postcode                                                                     
M1B            Scarborough                                      Rouge,Malvern
M1C            Scarborough               Rouge Hill,Highland Creek,Port Union
M1E            Scarborough                    West Hill,Guildwood,Morningside
M1G            Scarborough                                             Woburn
M1H            Scarborough                                          Cedarbrae
M1J            Scarborough                                Scarborough Village
M1K            Scarborough          Kennedy Park,Ionview,East Birchmount Park
M1L            Scarborough                      Clairlea,Golden Mile,Oakridge
M1M            Scarborough      Cliffside,Cliffcrest,Scarborough Village West
M1N            Scarborough                         Birch Cliff,Cliffside West
M1P            Scarborough  Wexford Heights,Dorset Park,Scarboro

### The number of rows is 103, as seen above

### Cleaning notebook

For cleaning the notebook, we already followed the steps involving missing data by idenifying what was missing, dealing with it and correcting it. We did this by dropping the row of the boroughs' with no values and changing the neighborhoods with no values to the boroughs' values.
Correcting data format:
    I don't think this is needed, but we can do it just incase.

In [15]:
#dtype() will check the data type
df2nb.dtypes


Borough          object
Neighbourhood    object
dtype: object

All the data types are objects. The Postcode is already in alphabetical order.


In [17]:
#last cell of notebook use .shape to print numbers of rows of your data frame
df2nb.shape
#103 is rows, 2 is columns.

(103, 2)

# Lat and Longitude

In [18]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
print(geo_data)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          

In [20]:
#use values to combine the two tables
df2nb['Latitude']=geo_data['Latitude'].values
df2nb['Longitude']=geo_data['Longitude'].values
print(df2nb)

                   Borough                                      Neighbourhood  \
Postcode                                                                        
M1B            Scarborough                                      Rouge,Malvern   
M1C            Scarborough               Rouge Hill,Highland Creek,Port Union   
M1E            Scarborough                    West Hill,Guildwood,Morningside   
M1G            Scarborough                                             Woburn   
M1H            Scarborough                                          Cedarbrae   
M1J            Scarborough                                Scarborough Village   
M1K            Scarborough          Kennedy Park,Ionview,East Birchmount Park   
M1L            Scarborough                      Clairlea,Golden Mile,Oakridge   
M1M            Scarborough      Cliffside,Cliffcrest,Scarborough Village West   
M1N            Scarborough                         Birch Cliff,Cliffside West   
M1P            Scarborough  

# Exploring and clustering

In [23]:
client_id='QIKHDMPLGGV1KK1GTSCWWCY250SWEPWMNZOBCDYYVDNNFFST'
client_secrete='244KRPN0ZU221MCY3MKMWCYWL2G2NLNGWWNZZCRTZHVCLWA5'
version=20190826
#defining woburn_data
woburn_data= neighbourhoods[neighbourhoods['Borough'] == 'Woburn'].reset_index(drop=True)


#getting neighbourhood's name, and neighbourhood's lat and longitude values
woburn_data.loc[0,'Neighbourhood']
neighbourhood_latitude=woburn_data.loc[0, 'Latitude']
neighbourhood_longitude = woburn_data.loc[0, 'Longitude']

neighbourhood_name=woburn_data.loc[0, 'Neighbourhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

NameError: name 'neighbourhoods' is not defined

In [ ]:
#create and display URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version,
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
#send the get request and examine results
results=requests.get(url).json()
results